In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# housing data import and add bias ahead of matrix
housing = fetch_california_housing()
m, n = housing.data.shape

In [2]:
n_epochs = 100000
learning_rate = 0.01

In [3]:
scaled_housing_data = housing.data / housing.data.max(axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
#scaled_housing_data_plus_bias

In [4]:
## tensorflow model
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

## random support of theta
theta = tf.Variable(tf.random.normal([n+1, 1], -1.0, 1.0), name="theta")

### tensorflow 2.0, refer to https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate.ipynb and https://www.tensorflow.org/guide/effective_tf2
@tf.function
def train(theta, X, y):
    y_pred = tf.matmul(X, theta, name="preditions")
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    gradients = 2/m * tf.matmul(tf.transpose(X), error)
    theta.assign(theta - learning_rate * gradients)
    return mse

for epoch in range(n_epochs+1):
    mse = train(theta, X, y)
    if epoch % 10000 == 0:
        print("Epoch {}, MSE = {}".format(epoch, mse.numpy()))

print("best beta : {}".format(theta))

Epoch 0, MSE = 57.39571762084961
Epoch 10000, MSE = 0.6467878222465515
Epoch 20000, MSE = 0.63975989818573
Epoch 30000, MSE = 0.6366565823554993
Epoch 40000, MSE = 0.6344910264015198
Epoch 50000, MSE = 0.6328314542770386
Epoch 60000, MSE = 0.6314508318901062
Epoch 70000, MSE = 0.630227267742157
Epoch 80000, MSE = 0.6290949583053589
Epoch 90000, MSE = 0.6280180811882019
Epoch 100000, MSE = 0.6269778609275818
best beta : <tf.Variable 'theta:0' shape=(9, 1) dtype=float32, numpy=
array([[-1.0595812 ],
       [ 6.370453  ],
       [ 0.88735497],
       [-0.45910722],
       [ 0.01300614],
       [ 0.47224718],
       [-2.1817088 ],
       [-3.208448  ],
       [ 3.5613496 ]], dtype=float32)>
